# Day 4 - Part 2: 모멘텀 전략 백테스트

## 학습 목표
- 모멘텀이 무엇인지 이해하기
- 12개월 수익률로 종목 순위 매기기
- 상위 5개 종목을 매월 선택하는 전략 구현하기

## 모멘텀이란?
"최근에 많이 오른 주식이 계속 오른다"는 전략입니다.
- 왜? 투자자들이 좋은 뉴스에 천천히 반응하기 때문
- 방법: 12개월 수익률이 높은 종목을 선택

## 1. 라이브러리 임포트

In [1]:
import backtrader as bt
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

print("라이브러리 임포트 완료!")

라이브러리 임포트 완료!


## 2. 코스피 200 종목 중 10개 선택 (실습용)

실제로는 더 많은 종목을 사용하지만, 처음엔 10개로 연습합니다.

In [2]:
# 실습용 10개 종목 (코스피 대표주)
tickers = {
    '005930.KS': '삼성전자',
    '000660.KS': 'SK하이닉스',
    '051910.KS': 'LG화학',
    '006400.KS': '삼성SDI',
    '035420.KS': 'NAVER',
    '005380.KS': '현대차',
    '012330.KS': '현대모비스',
    '028260.KS': '삼성물산',
    '068270.KS': '셀트리온',
    '207940.KS': '삼성바이오로직스'
}

print(f"총 {len(tickers)}개 종목 선택")
for ticker, name in tickers.items():
    print(f"  - {ticker}: {name}")

총 10개 종목 선택
  - 005930.KS: 삼성전자
  - 000660.KS: SK하이닉스
  - 051910.KS: LG화학
  - 006400.KS: 삼성SDI
  - 035420.KS: NAVER
  - 005380.KS: 현대차
  - 012330.KS: 현대모비스
  - 028260.KS: 삼성물산
  - 068270.KS: 셀트리온
  - 207940.KS: 삼성바이오로직스


## 3. 데이터 다운로드

2020년부터 2024년까지 데이터를 받습니다.

In [59]:
# 데이터 다운로드
start_date = '2019-01-01'  # 12개월 수익률 계산을 위해 1년 더 일찍 시작
end_date = '2024-12-31'

all_data = {}

print("데이터 다운로드 중...\n")

for ticker, name in tickers.items():
    try:
        data = yf.download(ticker, start=start_date, end=end_date, progress=False, multi_level_index=False)
        if not data.empty:
            all_data[ticker] = data
            print(f"{name}: {len(data)}일치 데이터")
        else:
            print(f"{name}: 데이터 없음")
    except Exception as e:
        print(f"{name}: 오류 - {e}")

print(f"\n총 {len(all_data)}개 종목 다운로드 완료!")

데이터 다운로드 중...



C:\Users\panis\AppData\Local\Temp\ipykernel_54764\4042869399.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date, end=end_date, progress=False, multi_level_index=False)


삼성전자: 1475일치 데이터


C:\Users\panis\AppData\Local\Temp\ipykernel_54764\4042869399.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date, end=end_date, progress=False, multi_level_index=False)


SK하이닉스: 1475일치 데이터


...

삼성물산: 1475일치 데이터


C:\Users\panis\AppData\Local\Temp\ipykernel_54764\4042869399.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date, end=end_date, progress=False, multi_level_index=False)


셀트리온: 1475일치 데이터


C:\Users\panis\AppData\Local\Temp\ipykernel_54764\4042869399.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date, end=end_date, progress=False, multi_level_index=False)


삼성바이오로직스: 1475일치 데이터

총 10개 종목 다운로드 완료!


## 4. 12개월 수익률 계산하기

각 종목의 12개월 수익률을 계산합니다.
- $\displaystyle 수익률 = \frac{(현재가 - 12개월전 가격)}{ 12개월전 가격 } * 100$

## 1. `freq` 에 들어가는 문자열(alias)

`freq` 인자는 사실상 **“어떤 DateOffset 을 쓸지 선택하는 문자열 별칭(alias)”**입니다.
공식 문서를 보면 `BusinessDay`, `MonthEnd`, `QuarterEnd` 같은 Offset 클래스들이 있고, 이들에 대해 `'B'`, `'M'`, `'Q'` 같은 문자열이 매핑되어 있습니다. 이걸 통틀어 **offset alias** 라고 부릅니다.

대표적인 것들만 적어보면:

* 일/영업일

  * `'D'` : calendar day (일 단위)
  * `'B'` : Business day (월~금)
  * `'C'` / `'CDay'` : CustomBusinessDay (사용자 정의 영업일)

* 주/월/분기/연

  * `'W'` : 주 단위
  * `'M'` : month end (월말)
  * `'MS'` : month start (월초)
  * `'BM'` / `'BMS'` : business month end/start
  * `'Q'`, `'QS'` : 분기말/분기초
  * `'BQ'`, `'BQS'` : business quarter end/start
  * `'A'`, `'Y'` : year end (연말)
  * `'AS'`, `'YS'` : year start (연초)
  * `'BA'`, `'BY'`, `'BAS'`, `'BYS'` : business year end/start

* 시/분/초 이하

  * `'H'` : 시간
  * `'T'`, `'min'` : 분
  * `'S'` : 초
  * `'L'`, `'ms'` : 밀리초
  * `'U'`, `'us'` : 마이크로초
  * `'N'` : 나노초

* 기타

  * `'BH'` : BusinessHour
  * `'SM'`, `'SMS'` : SemiMonthEnd/Begin
  * `'WOM-1MON'` 같은 WeekOfMonth류
  * 회계연도용 `'5253W-SUN'` 같은 특수 코드 등등

---

## 2. `pd.DateOffset(years=1)` 처럼 코드로 넣을 수 있는 경우

`pd.DateOffset` 은 “날짜에 더하거나, 특정 컴포넌트를 교체하는” 데 쓰는 객체인데,
키워드 인자 종류가 딱 정해져 있습니다.

### 2-1. “더하는(add)” 역할을 하는 인자들

이건 `Timedelta` 비슷하게 **기간을 더하는 용도**입니다:

* `years`
* `months`
* `weeks`
* `days`
* `hours`
* `minutes`
* `seconds`
* `milliseconds`
* `microseconds`
* `nanoseconds`

👉 총 **10개**

예:

```python
from pandas import DateOffset
ts = pd.Timestamp("2024-01-15")

ts + DateOffset(years=1)       # 1년 후
ts + DateOffset(months=3)      # 3개월 후
ts + DateOffset(weeks=2, days=1)
ts + DateOffset(hours=5, minutes=30)
```

### 2-2. “교체(replace)” 역할을 하는 인자들

이건 `datetime.replace(...)` 비슷하게 **특정 필드를 바꿔 끼우는 용도**입니다:

* `year`
* `month`
* `day`
* `weekday`
* `hour`
* `minute`
* `second`
* `microsecond`
* `nanosecond`

👉 총 **9개**

예:

```python
ts = pd.Timestamp("2024-01-15 09:10:11")

ts + DateOffset(day=1)      # 날짜를 "1일"로 교체 (2024-01-01 09:10:11)
ts + DateOffset(year=2020)  # 연도를 2020으로 교체
ts + DateOffset(hour=0)     # 시간을 0시로 교체
```

### 2-3. 그 밖의 인자

* `n`: 몇 번 반복할지 (기본 1)
* `normalize`: True면 결과를 날짜 00:00:00 에 맞춰서 자정으로 정규화할지

---

### 간단 정리

* `freq="..."` 문자열(alias)는

  * Offset 종류 + 별칭 조합으로 **수십 개(30~50개 이상)**
  * 자주 쓰는 건 `D, B, W, M, MS, Q, A, H, T, S` 정도.
* `pd.DateOffset(...)` 키워드는

  * 기간을 더하는 인자 10개 (`years`~`nanoseconds`)
  * 컴포넌트 교체 인자 9개 (`year`~`nanosecond`)
  * 나머지 보조 인자(`n`, `normalize`) 정도.


In [19]:
import pandas as pd

off = pd.DateOffset(years=1)
print(off, type(off))

<DateOffset: years=1> <class 'pandas._libs.tslibs.offsets.DateOffset'>


In [23]:
def calculate_12m_return(data):
    """
    12개월 수익률 계산
    """
    # 종가 데이터
    close = data['Close'].copy()
    
    # 252일 = 약 12개월 (영업일 기준)
    return_12m = close.pct_change(periods=252) * 100
    return return_12m

# 각 종목의 12개월 수익률 계산
returns_12m = {}

for ticker, data in all_data.items():
    returns_12m[ticker] = calculate_12m_return(data)

print("12개월 수익률 계산 완료!")

# 예시: 2023년 1월 1일 기준 수익률
example_date = '2023-01-02'
print(f"\n{example_date} 기준 12개월 수익률:")

for ticker in list(returns_12m.keys())[:5]:  # 처음 5개만 출력
    name = tickers[ticker]
    try:
        ret = returns_12m[ticker].loc[example_date]
        print(f"  {name}: {ret:.2f}%")
    except:
        print(f"  {name}: 데이터 없음")

12개월 수익률 계산 완료!

2023-01-02 기준 12개월 수익률:
  삼성전자: -26.86%
  SK하이닉스: -37.63%
  LG화학: -2.55%
  삼성SDI: -6.38%
  NAVER: -51.85%


In [24]:
returns_12m

{'005930.KS': Date
 2019-01-02          NaN
 2019-01-03          NaN
 2019-01-04          NaN
 2019-01-07          NaN
 2019-01-08          NaN
                 ...    
 2023-12-21    26.749816
 2023-12-22    30.866520
 2023-12-26    31.629535
 2023-12-27    34.669948
 2023-12-28    37.614784
 Name: Close, Length: 1231, dtype: float64,
 '000660.KS': Date
 2019-01-02          NaN
 2019-01-03          NaN
 2019-01-04          NaN
 2019-01-07          NaN
 2019-01-08          NaN
                 ...    
 2023-12-21    73.930912
 2023-12-22    77.527059
 2023-12-26    81.990462
 2023-12-27    80.351357
 2023-12-28    83.389350
 Name: Close, Length: 1231, dtype: float64,
 '051910.KS': Date
 2019-01-02          NaN
 2019-01-03          NaN
 2019-01-04          NaN
 2019-01-07          NaN
 2019-01-08          NaN
                 ...    
 2023-12-21   -22.134720
 2023-12-22   -22.561620
 2023-12-26   -22.722973
 2023-12-27   -21.419681
 2023-12-28   -19.062009
 Name: Close, Length: 1231, dt

In [26]:
def calculate_12m_return(data):
    """
    12개월 수익률 계산
    """
    # 종가 데이터
    close = data['Close'].copy()
    
    # 252일 = 약 12개월 (영업일 기준)
    return_12m = close.pct_change(freq=pd.DateOffset(years=1)) * 100
    
    return return_12m

# 각 종목의 12개월 수익률 계산
returns_12m = {} # Dict

for ticker, data in all_data.items():
    returns_12m[ticker] = calculate_12m_return(data)

print("12개월 수익률 계산 완료!")

# 예시: 2023년 1월 1일 기준 수익률
example_date = '2023-01-04'
print(f"\n{example_date} 기준 12개월 수익률:")

for ticker in list(returns_12m.keys())[:5]:  # 처음 5개만 출력
    name = tickers[ticker]
    try:
        ret = returns_12m[ticker].loc[example_date]
        print(f"  {name}: {ret:.2f}%")
    except:
        print(f"  {name}: 데이터 없음")

12개월 수익률 계산 완료!

2023-01-04 기준 12개월 수익률:
  삼성전자: -24.75%
  SK하이닉스: -36.12%
  LG화학: -5.17%
  삼성SDI: -6.94%
  NAVER: -49.84%


In [22]:
returns_12m

{'005930.KS': Date
 2019-01-02          NaN
 2019-01-03          NaN
 2019-01-04          NaN
 2019-01-07          NaN
 2019-01-08          NaN
                 ...    
 2023-12-21    32.213177
 2023-12-22    31.309386
 2023-12-26    35.266951
 2023-12-27    37.914991
 2023-12-28    41.592224
 Name: Close, Length: 1231, dtype: float64,
 '000660.KS': Date
 2019-01-02          NaN
 2019-01-03          NaN
 2019-01-04          NaN
 2019-01-07          NaN
 2019-01-08          NaN
                 ...    
 2023-12-21    82.404460
 2023-12-22    79.768586
 2023-12-26    85.299352
 2023-12-27    85.035780
 2023-12-28    88.203131
 Name: Close, Length: 1231, dtype: float64,
 '051910.KS': Date
 2019-01-02          NaN
 2019-01-03          NaN
 2019-01-04          NaN
 2019-01-07          NaN
 2019-01-08          NaN
                 ...    
 2023-12-21   -18.869407
 2023-12-22   -20.458668
 2023-12-26   -19.346788
 2023-12-27   -20.794048
 2023-12-28   -16.513916
 Name: Close, Length: 1231, dt

## 5. 매월 말일 찾기

매월 마지막 거래일에 종목을 교체합니다.

In [31]:
all_data['005930.KS'].index.to_period('M')

PeriodIndex(['2019-01', '2019-01', '2019-01', '2019-01', '2019-01', '2019-01',
             '2019-01', '2019-01', '2019-01', '2019-01',
             ...
             '2023-12', '2023-12', '2023-12', '2023-12', '2023-12', '2023-12',
             '2023-12', '2023-12', '2023-12', '2023-12'],
            dtype='period[M]', name='Date', length=1231)

In [ ]:
def get_month_end_dates(data, start, end):
    """
    매월 마지막 거래일 찾기
    """
    # 날짜 범위 필터링
    mask = (data.index >= start) & (data.index <= end)
    filtered = data[mask]
    
    # 월별로 그룹화하고 월별 마지막 날짜 추출
    month_ends = filtered.groupby(filtered.index.to_period('M')).apply(
        lambda x: x.index[-1]
    ).tolist()
    
    return month_ends

# 첫 번째 종목으로 월말 날짜 찾기
first_ticker = list(all_data.keys())[0]
rebalance_dates = get_month_end_dates(
    all_data[first_ticker],
    start='2020-01-01',
    end='2023-12-31'
)

print(f"총 {len(rebalance_dates)}번의 리밸런싱 날짜 확인")
print(f"\n처음 5개 날짜:")
for date in rebalance_dates[:5]:
    print(f"  - {date.strftime('%Y-%m-%d')}")

총 48번의 리밸런싱 날짜 확인

처음 5개 날짜:
  - 2020-01-31
  - 2020-02-28
  - 2020-03-31
  - 2020-04-29
  - 2020-05-29


In [32]:
rebalance_dates

[Timestamp('2020-01-31 00:00:00'),
 Timestamp('2020-02-28 00:00:00'),
 Timestamp('2020-03-31 00:00:00'),
 Timestamp('2020-04-29 00:00:00'),
 Timestamp('2020-05-29 00:00:00'),
 Timestamp('2020-06-30 00:00:00'),
 Timestamp('2020-07-31 00:00:00'),
 Timestamp('2020-08-31 00:00:00'),
 Timestamp('2020-09-29 00:00:00'),
 Timestamp('2020-10-30 00:00:00'),
 Timestamp('2020-11-30 00:00:00'),
 Timestamp('2020-12-30 00:00:00'),
 Timestamp('2021-01-29 00:00:00'),
 Timestamp('2021-02-26 00:00:00'),
 Timestamp('2021-03-31 00:00:00'),
 Timestamp('2021-04-30 00:00:00'),
 Timestamp('2021-05-31 00:00:00'),
 Timestamp('2021-06-30 00:00:00'),
 Timestamp('2021-07-30 00:00:00'),
 Timestamp('2021-08-31 00:00:00'),
 Timestamp('2021-09-30 00:00:00'),
 Timestamp('2021-10-29 00:00:00'),
 Timestamp('2021-11-30 00:00:00'),
 Timestamp('2021-12-30 00:00:00'),
 Timestamp('2022-01-28 00:00:00'),
 Timestamp('2022-02-28 00:00:00'),
 Timestamp('2022-03-31 00:00:00'),
 Timestamp('2022-04-29 00:00:00'),
 Timestamp('2022-05-

## 6. 상위 5개 종목 선택 함수

각 월말에 12개월 수익률이 높은 5개 종목을 선택합니다.

In [6]:
def select_top_n(returns_dict, date, n=5):
    """
    특정 날짜에 수익률 상위 N개 종목 선택
    
    Parameters:
    - returns_dict: {ticker: return_series} 딕셔너리
    - date: 선택할 날짜
    - n: 선택할 종목 수
    
    Returns:
    - 상위 N개 종목의 ticker 리스트
    """
    ranking = {}
    
    for ticker, returns in returns_dict.items():
        try:
            # 해당 날짜의 수익률 가져오기
            ret = returns.loc[date]
            # NaN이 아니면 추가
            if not np.isnan(ret):
                ranking[ticker] = ret
        except:
            continue
    
    # 수익률 기준 내림차순 정렬
    sorted_tickers = sorted(ranking.items(), key=lambda x: x[1], reverse=True)
    
    # 상위 N개 티커만 반환
    top_n = [ticker for ticker, ret in sorted_tickers[:n]]
    
    return top_n, ranking

# 예시: 2023년 1월 31일 기준 상위 5개
example_date = pd.Timestamp('2023-01-31')
top_5, all_returns = select_top_n(returns_12m, example_date, n=5)

print(f"{example_date.strftime('%Y-%m-%d')} 선택 종목:\n")
for i, ticker in enumerate(top_5, 1):
    name = tickers[ticker]
    ret = all_returns[ticker]
    print(f"  {i}. {name}: {ret:.2f}%")

2023-01-31 선택 종목:

  1. 삼성물산: 8.85%
  2. LG화학: 7.38%
  3. 삼성SDI: 4.10%
  4. 셀트리온: 2.17%
  5. 삼성바이오로직스: -1.66%


## 7. 모멘텀 전략 구현 (Step by Step)

이제 Backtrader로 전략을 구현합니다.

In [7]:
class MomentumStrategy(bt.Strategy):
    """
    모멘텀 전략
    - 매월 말: 12개월 수익률 상위 5개 선택
    - 동일 가중으로 투자 (각 20%)
    """
    
    params = (
        ('rebalance_dates', None),  # 리밸런싱 날짜 리스트
        ('top_n', 5),  # 선택할 종목 수
    )
    
    def __init__(self):
        # 각 종목의 12개월 수익률 저장
        self.returns_12m = {}
        for d in self.datas:
            # 252일 전 가격 대비 수익률
            self.returns_12m[d._name] = (d.close / d.close(-252) - 1) * 100
        
        self.rebalance_dates = [pd.Timestamp(d) for d in self.params.rebalance_dates]
        self.next_rebalance_idx = 0
        
        print("전략 초기화 완료!")
    
    def next(self):
        """
        매일 실행되는 함수
        """
        current_date = pd.Timestamp(self.datas[0].datetime.date(0))
        
        # 리밸런싱 날짜인지 확인
        if self.next_rebalance_idx < len(self.rebalance_dates):
            next_rebalance = self.rebalance_dates[self.next_rebalance_idx]
            
            if current_date >= next_rebalance:
                self.rebalance()
                self.next_rebalance_idx += 1
    
    def rebalance(self):
        """
        포트폴리오 리밸런싱
        """
        current_date = self.datas[0].datetime.date(0)
        
        # Step 1: 각 종목의 12개월 수익률 계산
        ranking = {}
        for d in self.datas:
            try:
                ret = self.returns_12m[d._name][0]
                if not np.isnan(ret):
                    ranking[d._name] = (d, ret)
            except:
                continue
        
        # Step 2: 상위 N개 선택
        sorted_stocks = sorted(ranking.items(), key=lambda x: x[1][1], reverse=True)
        top_n = [item[1][0] for item in sorted_stocks[:self.params.top_n]]
        
        # Step 3: 기존 포지션 정리
        for d in self.datas:
            if self.getposition(d).size > 0:
                if d not in top_n:
                    self.close(d)
        
        # Step 4: 선택된 종목에 동일 가중 투자
        target_value = self.broker.getvalue() / self.params.top_n
        
        for d in top_n:
            current_value = self.getposition(d).size * d.close[0]
            target_shares = int(target_value / d.close[0])
            
            # 목표 수량과 현재 수량 차이만큼 매매
            diff = target_shares - self.getposition(d).size
            
            if diff > 0:
                self.buy(data=d, size=diff)
            elif diff < 0:
                self.sell(data=d, size=-diff)
        
        # 선택된 종목 출력
        print(f"\n{current_date} 리밸런싱:")
        for i, d in enumerate(top_n, 1):
            ret = ranking[d._name][1]
            print(f"  {i}. {d._name}: {ret:.2f}%")

## 8. 백테스트 실행 준비

In [8]:
# Cerebro 엔진 생성
cerebro = bt.Cerebro()

# 초기 자금 (1억원)
cerebro.broker.setcash(100_000_000)

# 수수료 (0.1%)
cerebro.broker.setcommission(commission=0.001)

print("Cerebro 엔진 생성 완료!")

Cerebro 엔진 생성 완료!


## 9. 데이터 추가하기

In [9]:
# 각 종목 데이터를 Cerebro에 추가
for ticker, data in all_data.items():
    # Backtrader용 데이터 피드 생성
    data_bt = bt.feeds.PandasData(
        dataname=data,
        name=ticker,  # 종목 이름 지정
        fromdate=datetime(2020, 1, 1),
        todate=datetime(2023, 12, 31)
    )
    cerebro.adddata(data_bt)

print(f"{len(all_data)}개 종목 데이터 추가 완료!")

10개 종목 데이터 추가 완료!


## 10. 전략 추가 및 실행

In [10]:
# 전략 추가
cerebro.addstrategy(
    MomentumStrategy,
    rebalance_dates=rebalance_dates,
    top_n=5
)

# 분석기 추가
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')

print("전략 및 분석기 추가 완료!")

전략 및 분석기 추가 완료!


## 11. 백테스트 실행!

In [11]:
# 시작 전 자금
start_value = cerebro.broker.getvalue()

print(f"\n{'='*60}")
print(f"모멘텀 전략 백테스트 시작!")
print(f"{'='*60}")
print(f"시작 자금: {start_value:,.0f}원")
print(f"종목 수: {len(all_data)}개")
print(f"선택: 상위 5개")
print(f"리밸런싱: 매월 말 ({len(rebalance_dates)}회)")
print(f"{'='*60}\n")

# 실행
results = cerebro.run()
strat = results[0]

# 최종 결과
end_value = cerebro.broker.getvalue()
profit = end_value - start_value
return_pct = (profit / start_value) * 100

print(f"\n{'='*60}")
print(f"백테스트 종료!")
print(f"{'='*60}")
print(f"최종 자금: {end_value:,.0f}원")
print(f"수익금: {profit:,.0f}원")
print(f"총 수익률: {return_pct:.2f}%")

# 분석 결과
sharpe = strat.analyzers.sharpe.get_analysis()
dd = strat.analyzers.drawdown.get_analysis()
returns = strat.analyzers.returns.get_analysis()

print(f"\n성과 지표:")
print(f"  - 연간 수익률: {returns['rnorm']:.2f}%")
print(f"  - 샤프 비율: {sharpe.get('sharperatio', 0):.2f}")
print(f"  - 최대 낙폭(MDD): {dd['max']['drawdown']:.2f}%")
print(f"{'='*60}\n")


모멘텀 전략 백테스트 시작!
시작 자금: 100,000,000원
종목 수: 10개
선택: 상위 5개
리밸런싱: 매월 말 (48회)



전략 초기화 완료!

2021-01-08 리밸런싱:
  1. 051910.KS: 222.11%
  2. 006400.KS: 219.09%
  3. 005380.KS: 111.83%
  4. 068270.KS: 107.97%
  5. 207940.KS: 95.33%

2021-01-11 리밸런싱:
  1. 051910.KS: 224.89%
  2. 006400.KS: 219.34%
  3. 005380.KS: 134.31%
  4. 068270.KS: 114.92%
  5. 207940.KS: 100.47%

2021-01-12 리밸런싱:
  1. 006400.KS: 224.48%
  2. 051910.KS: 217.25%
  3. 005380.KS: 128.62%
  4. 068270.KS: 126.40%
  5. 207940.KS: 98.54%

2021-01-13 리밸런싱:
  1. 006400.KS: 226.27%
  2. 051910.KS: 225.54%
  3. 005380.KS: 127.85%
  4. 068270.KS: 118.61%
  5. 207940.KS: 98.09%

2021-01-14 리밸런싱:
  1. 051910.KS: 238.60%
  2. 006400.KS: 227.65%
  3. 005380.KS: 127.26%
  4. 068270.KS: 107.83%
  5. 207940.KS: 99.76%

2021-01-15 리밸런싱:
  1. 051910.KS: 223.39%
  2. 006400.KS: 217.54%
  3. 005380.KS: 118.71%
  4. 207940.KS: 91.43%
  5. 068270.KS: 89.59%

2021-01-18 리밸런싱:
  1. 051910.KS: 200.30%
  2. 006400.KS: 184.03%
  3. 005380.KS: 115.75%
  4. 207940.KS: 80.53%
  5. 068270.KS: 80.44%

2021-01-19 리밸런싱:
  1. 051910.K

## 12. 결과 시각화

- matplotlib 이미지 처리 방식중에 1가지 골라서 출력하기

In [56]:
print("interactive:", plt.isinteractive())
print("backend:", plt.get_backend())

interactive: True
backend: TkAgg


In [57]:
# backend 를 TkAgg 로 실행
# 외부 이미지 창을 닫아야 다음 셀로 실행이 넘어감
%matplotlib tk
plt.ioff()
cerebro.plot(style='candlestick', iplot=False, use="TkAgg")

[[<Figure size 640x480 with 22 Axes>]]

In [58]:
print("interactive:", plt.isinteractive())
print("backend:", plt.get_backend())

interactive: False
backend: TkAgg


## 13. 생각해볼 질문들

1. 왜 12개월 수익률을 사용할까요? 3개월이나 24개월은 안 될까요?
2. 종목을 5개만 선택하는 이유는? 1개나 20개는 어떨까요?
3. 수수료가 0.1%에서 0.25%로 올라가면 수익률은 얼마나 차이날까요?

## 정리

배운 내용:
- 모멘텀 = 최근에 오른 주식이 계속 오른다
- 12개월 수익률로 종목 순위 매기기
- 매월 상위 5개 종목 선택하여 리밸런싱
- Backtrader로 전략 구현 및 백테스트

주의사항:
- 과거 결과가 좋다고 미래도 좋은 건 아님
- 수수료와 슬리피지 고려 필수
- 시장 환경이 바뀌면 전략도 바뀔 수 있음